# Model Fitting to predict Seasonal Flu Vaccination: XGBoost

There is one section with a gridsearch in it that I will convert to *markdown cells* so that the cells are not run when the notebook is run. The gridsearch cells are followed by cells with an XGBoost model rerun with the best parameters

Three models were run. The first one is not shown in the notebook. The fiindings are summarized below: 

Model|Accuracy|Precision|Recall|AUC|
-------------------------|--------|---------|------|---|
Without switching classes|0.784|0.784|0.739|0.781|
Switching classes|0.797|0.800|0.825|0.794|
Add feat. engineering|0.794|0/798|0.822|0.791|

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, ConfusionMatrixDisplay

C:\ProgramData\Anaconda3\envs\xgboost\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### **Import Data**

In [2]:
data = pd.read_csv('../data/train_clean.csv')

**Switch labels so that vaccinated is 0 and unvaccinated is 1**

In [3]:
data['h1n1_vaccine'].value_counts(normalize=True)

0    0.787546
1    0.212454
Name: h1n1_vaccine, dtype: float64

In [4]:
data['seasonal_vaccine'].value_counts(normalize=True)

0    0.534392
1    0.465608
Name: seasonal_vaccine, dtype: float64

In [5]:
# since we're interested in the people who did not get vaccinated: switch 0 and 1 labels
data['h1n1_vaccine'].replace({1:0, 0:1}, inplace=True)
data['h1n1_vaccine'].value_counts()

1    21033
0     5674
Name: h1n1_vaccine, dtype: int64

In [6]:
data['seasonal_vaccine'].replace({1:0, 0:1}, inplace=True)
data['seasonal_vaccine'].value_counts()

1    14272
0    12435
Name: seasonal_vaccine, dtype: int64

In [7]:
data.columns

Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation', 'h1n1_vaccine', 'seasonal_vaccine'],
      dtype='object')

In [8]:
categorical_columns = list(data.select_dtypes('object').columns)
categorical_columns

['doctor_recc_h1n1',
 'doctor_recc_seasonal',
 'chronic_med_condition',
 'health_worker',
 'health_insurance',
 'age_group',
 'education',
 'race',
 'sex',
 'income_poverty',
 'marital_status',
 'rent_or_own',
 'employment_status',
 'hhs_geo_region',
 'census_msa',
 'employment_industry',
 'employment_occupation']

### **Define X and y for Model**

In [9]:
X = data.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])
y = data['seasonal_vaccine']

In [10]:
Xd = pd.get_dummies(data=X, columns=categorical_columns, drop_first=True)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(Xd, y, random_state=42, stratify=y)

In [12]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [13]:
y_train = np.array(y_train)
y_test = np.array(y_test)

## Run GridSearch to find best paramaters

**Please convert the cells in this section from markdown to code if you want to run the gridsearch**

```
xg_cls = xgb.XGBClassifier(random_state=42, use_label_encoder=False, n_jobs=-1, eval_metric='error')

params = {
    'n_estimators' : [100, 250],
    'max_depth' : [3, 5, 6],
    'learning_rate' : [0.01, 0.1],
    'colsample_bytree' : [0.5, 1],
    'subsample' : [0.6, 1] 
}

gs = GridSearchCV(estimator=xg_cls, param_grid=params, scoring='roc_auc')

gs.fit(X_train, y_train)

y_pred = gs.predict(X_test)
```
--------------

```
gs.best_score_

# result 0.8602725008085251
```
-----------

```
gs.best_estimator_

 # result
    XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5,
              enable_categorical=False, eval_metric='error', gamma=0, gpu_id=-1,
              importance_type=None, interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=-1, num_parallel_tree=1,
              predictor='auto', random_state=42, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)
```
------------------

```
gs.best_params_

# result

{'colsample_bytree': 0.5,
 'learning_rate': 0.1,
 'max_depth': 5,
 'n_estimators': 100,
 'subsample': 1}
 
 ```

```
accuracy_score(y_test, y_pred), accuracy_score(y_train, gs.predict(X_train))

precision_score(y_test, y_pred), precision_score(y_train, gs.predict(X_train))

recall_score(y_test, y_pred), recall_score(y_train, gs.predict(X_train))

roc_auc_score(y_test, y_pred), roc_auc_score(y_train, gs.predict(X_train))


ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=['Vaccinated', 'Unvaccinated'])

```

Model|Accuracy|Precision|Recall|AUC|
-------------------------|--------|---------|------|---|
Without switching classes|0.784|0.784|0.739|0.781|
Switching classes|0.797|0.800|0.825|0.794|

**Run best model with optimized parameters from gridsearch**

In [14]:
xg_cls = xgb.XGBClassifier(random_state=42,
                           use_label_encoder=False,
                           max_depth=5,
                           colsample_bytree = 0.5,
                           subsample=1,
                           learning_rate=0.1,
                           n_estimators=100,
                           n_jobs=-1, 
                           eval_metric='error'
                            )

xg_cls.fit(X_train, y_train)

y_pred = xg_cls.predict(X_test)

In [15]:
accuracy_score(y_test, y_pred), accuracy_score(y_train, xg_cls.predict(X_train))

(0.7970645499475812, 0.809735396904643)

In [16]:
precision_score(y_test, y_pred), precision_score(y_train, xg_cls.predict(X_train))

(0.8005976636783483, 0.8115339419687245)

In [17]:
recall_score(y_test, y_pred), recall_score(y_train, xg_cls.predict(X_train))

(0.8259529147982063, 0.8387518684603886)

In [18]:
roc_auc_score(y_test, y_pred), roc_auc_score(y_train, xg_cls.predict(X_train))

(0.7949320701363177, 0.8075916751695037)

Model|Accuracy|Precision|Recall|AUC|
-------------------------|--------|---------|------|---|
Without switching classes|0.784|0.784|0.739|0.781|
Switching classes|0.797|0.800|0.825|0.794|


**Get top 15 best features**

In [19]:
top_feat = pd.DataFrame({'feature': Xd.columns, 'importance':xg_cls.feature_importances_}).sort_values('importance', ascending=False).head(20)

In [20]:
top_feat

,feature,importance
21,doctor_recc_seasonal_1.0,0.118557
14,opinion_seas_vacc_effective,0.105583
15,opinion_seas_risk,0.077143
32,age_group_65+ Years,0.056827
28,health_insurance_yes,0.026104
26,health_worker_yes,0.025488
25,health_worker_no_response,0.020186
31,age_group_55 - 64 Years,0.017746
46,rent_or_own_Rent,0.015571
65,employment_industry_fcxhlnwr,0.015452


In [22]:
top_feat.to_csv('../data/XGB_seasonal_feature_imp.csv', index=False)

### Rerun Model with Feature Engineering
Such as make a new feature that is 1 if the subject had both h1n1_concern and h1n1_knowledge, another one for all the behaviors and a third one for doctor recommended

**Feature Engineering** :

In [24]:
#####
data['doctor_recc_h1n1'].value_counts()

0.0            19139
1.0             5408
no_response     2160
Name: doctor_recc_h1n1, dtype: int64

In [25]:
data['doctor_recc_h1n1'].dtypes

dtype('O')

In [26]:
data['doctor_recc_h1n1'] = data['doctor_recc_h1n1'].replace({'0.0': 1, '1.0': 2, 'no_response': 0})

data['doctor_recc_h1n1'].value_counts()

1    19139
2     5408
0     2160
Name: doctor_recc_h1n1, dtype: int64

In [27]:
#####
data['doctor_recc_seasonal'].value_counts()

0.0            16453
1.0             8094
no_response     2160
Name: doctor_recc_seasonal, dtype: int64

In [28]:
data['doctor_recc_seasonal'] = data['doctor_recc_seasonal'].replace({'0.0': 1, '1.0': 2, 'no_response': 0})
data['doctor_recc_seasonal'].value_counts()

1    16453
2     8094
0     2160
Name: doctor_recc_seasonal, dtype: int64

In [29]:
#####
data['chronic_med_condition'].value_counts()

0.0            18446
1.0             7290
no_response      971
Name: chronic_med_condition, dtype: int64

In [30]:
data['chronic_med_condition'] = data['chronic_med_condition'].replace({'no_response' : 0, '0.0' : 1, '1.0': 2})
data['chronic_med_condition'].value_counts()

1    18446
2     7290
0      971
Name: chronic_med_condition, dtype: int64

In [31]:
#####
data['age_group'].value_counts()

65+ Years        6843
55 - 64 Years    5563
45 - 54 Years    5238
18 - 34 Years    5215
35 - 44 Years    3848
Name: age_group, dtype: int64

In [32]:
data['age_group'].replace({
    '18 - 34 Years' : 0,
    '35 - 44 Years' : 1,
    '45 - 54 Years' : 2,
    '55 - 64 Years' : 3,
    '55 - 64 Years' : 4,
    '65+ Years' : 5
    
}, inplace=True)

data['age_group'].value_counts()

5    6843
4    5563
2    5238
0    5215
1    3848
Name: age_group, dtype: int64

In [33]:
#####

data['health_worker'].value_counts()

no             23004
yes             2899
no_response      804
Name: health_worker, dtype: int64

In [34]:
data['health_worker'].replace({'no_response': 0,'no': 1, 'yes': 2}, inplace=True)
data['health_worker'].value_counts()

1    23004
2     2899
0      804
Name: health_worker, dtype: int64

In [35]:
# make a new feature h1n1_all for participants who had both h1n1_concern and h1n1_knowledge
data[['h1n1_concern', 'h1n1_knowledge']].dtypes

h1n1_concern      float64
h1n1_knowledge    float64
dtype: object

In [36]:
data['h1n1_all'] = data['h1n1_concern'] * data['h1n1_knowledge']

In [37]:
# make a new feature h1n1_all for participants who responded yes to all the behavior questions
behaviour_columns = [column for column in list(data.columns) if 'behavioral' in column]
data[behaviour_columns].dtypes

behavioral_antiviral_meds      float64
behavioral_avoidance           float64
behavioral_face_mask           float64
behavioral_wash_hands          float64
behavioral_large_gatherings    float64
behavioral_outside_home        float64
behavioral_touch_face          float64
dtype: object

In [38]:
data['behavioral_all'] = 1

for column in behaviour_columns:
    data['behavioral_all'] = data['behavioral_all'] * data[column]

In [39]:
# make a new feature opinion_all for participants based on their opinion scores to the opinion questions
opinion_columns = [column for column in list(data.columns) if 'opinion' in column]
data[opinion_columns].dtypes

opinion_h1n1_vacc_effective    float64
opinion_h1n1_risk              float64
opinion_h1n1_sick_from_vacc    float64
opinion_seas_vacc_effective    float64
opinion_seas_risk              float64
opinion_seas_sick_from_vacc    float64
dtype: object

In [40]:
data['opinion_all'] = 1

for column in opinion_columns:
    data['opinion_all'] = data['opinion_all'] * data[column]

In [41]:
# doctor recommended columns
doctor_columns = [column for column in list(data.columns) if 'doctor' in column]

In [42]:
doctor_columns

['doctor_recc_h1n1', 'doctor_recc_seasonal']

In [43]:
data['doctor_recc_all'] = 1

for column in opinion_columns:
    data['doctor_recc_all'] = data['doctor_recc_all'] * data[column]

In [44]:
# health_worker and age group

data['health_worker_by_age'] = data['health_worker'] * data['age_group']

In [45]:
data.columns

Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation', 'h1n1_vaccine', 'seasonal_vaccine', 'h1n1_all',
       'behavioral_all', 'opinion_all', 'doctor_recc_all',
       'health_worker_by_age'],
   

In [46]:
X = data.drop(columns=['seasonal_vaccine', 'h1n1_vaccine'])
y = data['seasonal_vaccine']

In [47]:
Xd = pd.get_dummies(data=X, columns=categorical_columns, drop_first=True)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(Xd, y, random_state=42, stratify=y)

In [49]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [50]:
y_train = np.array(y_train)
y_test = np.array(y_test)

**Run best model on this new set of features**

In [51]:
xg_feat = xgb.XGBClassifier(random_state=42,
                           use_label_encoder=False,
                           max_depth=5,
                           colsample_bytree = 0.5,
                           subsample=1,
                           learning_rate=0.1,
                           n_estimators=100,
                           n_jobs=-1, 
                           eval_metric='error'
                            )

xg_feat.fit(X_train, y_train)

y_pred = xg_feat.predict(X_test)

In [52]:
accuracy_score(y_test, y_pred), accuracy_score(y_train, xg_feat.predict(X_train))

(0.7940691927512356, 0.8096854717923115)

In [53]:
precision_score(y_test, y_pred), precision_score(y_train, xg_feat.predict(X_train))

(0.7983673469387755, 0.8117423557083409)

In [54]:
recall_score(y_test, y_pred), recall_score(y_train, xg_feat.predict(X_train))

(0.8223094170403588, 0.8382847533632287)

In [55]:
roc_auc_score(y_test, y_pred), roc_auc_score(y_train, xg_feat.predict(X_train))

(0.7919845573461685, 0.8075725718349491)

Model|Accuracy|Precision|Recall|AUC|
-------------------------|--------|---------|------|---|
Without switching classes|0.784|0.784|0.739|0.781|
Switching classes|0.797|0.800|0.825|0.794|
Add feat. engineering|0.794|0/798|0.822|0.791|

**Get top 15 best features**

In [56]:
top_feat = pd.DataFrame({'feature': Xd.columns, 'importance':xg_feat.feature_importances_}).sort_values('importance', ascending=False).head(15)

In [57]:
top_feat

,feature,importance
27,doctor_recc_seasonal_2,0.117716
14,opinion_seas_vacc_effective,0.101134
15,opinion_seas_risk,0.075236
37,age_group_5,0.038628
23,health_worker_by_age,0.035109
26,doctor_recc_seasonal_1,0.028652
33,health_insurance_yes,0.018157
31,health_worker_2,0.017885
70,employment_industry_fcxhlnwr,0.016348
51,rent_or_own_Rent,0.014340
